In [1]:
from __future__ import division
from api import git_access,api_access
from git_log import git2repo
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import networkx as nx
import re
from git_log import git2data,git_commit_info
import threading
from threading import Barrier
from multiprocessing import Queue
from os.path import dirname as up
import os
import platform

In [2]:
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    data_path = os.getcwd() + '/Test_projects.csv'
else:
    data_path = os.getcwd() + '\\Test_projects.csv'
project_list = pd.read_csv(data_path)
for i in range(project_list.shape[0]):
    try:
        print("I am here")
        access_token = project_list.loc[i,'access_token']
        repo_owner = project_list.loc[i,'repo_owner']
        source_type = project_list.loc[i,'source_type']
        git_url = project_list.loc[i,'git_url']
        api_base_url = project_list.loc[i,'api_base_url']
        repo_name = project_list.loc[i,'repo_name']
        #git_data = git2data.git2data(access_token,repo_owner,source_type,git_url,api_base_url,repo_name)
        git_data = git_commit_info.git2data(access_token,repo_owner,source_type,git_url,api_base_url,repo_name)
        git_data.create_data()
    except ValueError as e:
        print("Exception occured for ",project_list.loc[i,'git_url'])
        print(e)

I am here
http://api.github.com/repos/jenkinsci/github-plugin/issues?state=all&page=1&per_page=100
http://api.github.com/repos/jenkinsci/github-plugin/issues?state=all&page=2&per_page=100
http://api.github.com/repos/jenkinsci/github-plugin/issues?state=all&page=3&per_page=100
http://api.github.com/repos/jenkinsci/github-plugin/releases?page=1&per_page=100
http://api.github.com/repos/jenkinsci/github-plugin/issues/events?page=1&per_page=100
http://api.github.com/repos/jenkinsci/github-plugin/issues/events?page=2&per_page=100
http://api.github.com/repos/jenkinsci/github-plugin/issues/events?page=3&per_page=100
http://api.github.com/repos/jenkinsci/github-plugin/issues/events?page=4&per_page=100
http://api.github.com/repos/jenkinsci/github-plugin/issues/events?page=5&per_page=100
http://api.github.com/repos/jenkinsci/github-plugin/issues/events?page=6&per_page=100
http://api.github.com/repos/jenkinsci/github-plugin/issues/events?page=7&per_page=100
http://api.github.com/repos/jenkinsci/gi

In [3]:
df = pd.read_pickle('/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/release/github-plugin_release.pkl')



In [4]:
df

,Release_id,author_logon,tag,created_at
0,20529732,KostyaSha,v1.29.4,2019-02-17T15:58:01Z
1,13299419,lanwen,v1.29.3,2018-10-08T09:55:38Z
2,13299183,lanwen,v1.29.2,2018-06-15T12:12:29Z
3,13299172,lanwen,v1.29.1,2018-05-30T12:08:12Z
4,9261251,lanwen,v1.29.0,2018-01-16T23:09:35Z
5,8212981,lanwen,v1.28.1,2017-10-22T20:41:45Z
6,7282839,lanwen,v1.28.0,2017-08-04T10:02:15Z
7,6108070,lanwen,v1.27.0,2017-04-18T12:04:37Z
8,5954829,lanwen,v1.26.2,2017-04-03T10:44:48Z
9,5575080,lanwen,v1.26.1,2017-02-26T11:42:58Z
